In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-13 02:35:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.23MB/s    in 0.7s    

2021-12-13 02:35:05 (1.23 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
dataset = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + dataset
spark.sparkContext.addFile(url)
office_products_df = spark.read.csv(SparkFiles.get(dataset), sep="\t", header=True, inferSchema=True)

# Show DataFrame
office_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [6]:
# Count the number of records (rows) in the dataset
print("Number of records: " + str(office_products_df.count()))

Number of records: 2642434


In [7]:
# Transform the dataset to fit the tables in the schema
from pyspark.sql.functions import to_date

review_id_table = office_products_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-mm-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R18RVCKGH1SSI9|   43081963|B001BM2MAC|     307809868| 2015-01-31|
|R3L4L6LW1PUOFY|   10951564|B00DZYEXPQ|      75004341| 2015-01-31|
|R2J8AWXWTDX2TF|   21143145|B00RTMUHDW|     529689027| 2015-01-31|
|R1PR37BR7G3M6A|   52782374|B00D7H8XB6|     868449945| 2015-01-31|
|R3BDDDZMZBZDPU|   24045652|B001XCWP34|      33521401| 2015-01-31|
| R8T6MO75ND212|   21751234|B004J2NBCO|     214932869| 2015-01-31|
|R2YWMQT2V11XYZ|    9109358|B00MOPAG8K|     863351797| 2015-01-31|
|R1V2HYL6OI9V39|    9967215|B003AHIK7U|     383470576| 2015-01-31|
|R3BLQBKUNXGFS4|   11234247|B006TKH2RO|     999128878| 2015-01-31|
|R17MOWJCAR9Y8Q|   12731488|B00W61M9K0|     622066861| 2015-01-31|
|R11EPG1GHOXMGB|   49861762|B00BXOGI3A|     688569009| 2015-01-31|
|R2797NKTEX5THN|    1541556|B0018RHWGE|     513803406| 2015-01

In [16]:
products = office_products_df.select(["product_id", "product_title"])
products = products.drop_duplicates(subset=['product_id'])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0007262973|Pastels (Collins ...|
|0113000316|123GetInk -14-pac...|
|0136039847|Pearson MyHistory...|
|0307382214|Happy Baby, Healt...|
|0395742889|Webster's II New ...|
|0439165210|Alpha Tales Learn...|
|043928631X|Harry Potter Lent...|
|0439394058|Scholastic SC9394...|
|0439492092|Scholastic Teache...|
|043949219X|Scholastic ¡Color...|
|0439492335|Scholastic TF3078...|
|043949284X|Scholastic Teache...|
|0439492874|Scholastic Teache...|
|0439492882|Scholastic Teache...|
|0439492890|Scholastic Teache...|
|0439492904|Scholastic Teache...|
|0439499488|Scholastic Manusc...|
|043949950X|Scholastic Alphab...|
|043949964X|Scholastic Daily ...|
|0439499798|Colors and Shapes...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
customers = office_products_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed('count(customer_id)','customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43622307|             1|
|   33972406|             1|
|     108460|             1|
|   43515569|             1|
|     132406|             1|
|   42560427|             1|
|    1673863|             2|
|   14552054|             1|
|   45632184|             1|
|   14703850|             2|
|   35768925|             3|
|   45640757|            28|
|   33194403|             1|
|   39383689|             1|
|   10645124|             1|
|   52512151|            10|
|   14646714|             1|
|   38659596|             1|
|   15634680|             1|
|   44178035|             1|
+-----------+--------------+
only showing top 20 rows



In [26]:
vine_table = office_products_df[["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]]

# Convert string to integer
from pyspark.sql.types import IntegerType
vine_table = vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))

vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R18RVCKGH1SSI9|          5|            0|          0|   N|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|
|R1PR37BR7G3M6A|          1|            2|          3|   N|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|
| R8T6MO75ND212|          5|            0|          0|   N|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|
|R11EPG1GHOXMGB|          4|            0|          0|   N|
|R2797NKTEX5THN|          5|            0|          0|   N|
|R2RUTMJS50E4WF|          5|            0|          0|   N|
|R1FZYL1HX7O4RM|          1|            

In [18]:
# Load the DataFrames that correspond to tables into an RDS instance
mode = "append"
jdbc_url = "jdbc:postgresql://<amazon-endpoint-address>:5432/<amazon-db-name>"
config = {
    "user" : "username", 
    "password" : "password",
    "driver" : "org.postgresql.Driver"
}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [28]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)